In [1]:
from google.colab import drive

# 데이터셋 pet.zip 구글드라이브에 업로드한 뒤
# 구글 드라이브 마운트: 모든 액세스 권한 허용해야 함
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd dataset/ # 압축을 풀고 싶은 위치, 미리 폴더 생성해둬야 함
!unzip -qq "/content/drive/MyDrive/pet.zip" # 압축파일이 있는 경로

In [16]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import models
from torchvision import transforms
from torchvision.datasets import ImageFolder

# 하이퍼파라미터들 선언
hyperparams = {
    "batch_size": 4,
    "learning_rate": 0.0001,
    "epochs": 5,
    "transform": transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48235, 0.45882, 0.40784],
                std=[1.0 / 255.0, 1.0 / 255.0, 1.0 / 255.0],
            ),
        ]
    ),
}

# tran, test 데이터셋, 데이터 로더 선언
train_dataset = ImageFolder("/content/dataset/train", transform=hyperparams["transform"])
test_dataset = ImageFolder("/content/dataset/test", transform=hyperparams["transform"])

train_dataloader = DataLoader(train_dataset, batch_size=hyperparams["batch_size"], shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparams["batch_size"], shuffle=True, drop_last=True)

# 모델 선언, ImageNet 데이터셋에 pretrain된 VGG-16 모델 사용
model = models.vgg16(weights="VGG16_Weights.IMAGENET1K_V1")
model.classifier[6] = nn.Linear(4096, len(train_dataset.classes))

# device, 손실함수, optimizer 선언
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=hyperparams["learning_rate"])

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 132MB/s] 


In [17]:
# 학습 코드
for epoch in range(hyperparams["epochs"]):
    cost = 0.0

    for images, classes in train_dataloader:
        images = images.to(device)
        classes = classes.to(device)

        output = model(images)
        loss = criterion(output, classes)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :    1, Cost : 0.284
Epoch :    2, Cost : 0.092
Epoch :    3, Cost : 0.061
Epoch :    4, Cost : 0.041
Epoch :    5, Cost : 0.027


In [18]:
# 테스트 코드
with torch.no_grad():
    model.eval()

    accuracy = 0.0
    for images, classes in test_dataloader:
        images = images.to(device)
        classes = classes.to(device)

        outputs = model(images)
        probs = F.softmax(outputs, dim=-1)
        outputs_classes = torch.argmax(probs, dim=-1)

        accuracy += int(torch.eq(classes, outputs_classes).sum())

    print(f"acc@1 : {accuracy / (len(test_dataloader) * hyperparams['batch_size']) * 100:.2f}%")

acc@1 : 97.38%


In [19]:
# 모델 저장
torch.save(model.state_dict(), "/content/VGG16.pt")
print("Saved the model weights")

Saved the model weights
